In [40]:
library(data.table)

In [56]:
covar <- fread('/nfs/research/birney/projects/association/gene_burden/covariates//ukbb_200k_covar_v1.csv')
geno <- fread('/nfs/research/birney/users/shimin/ukbb/wes/200k/gene_modules/data/module_burden_m1_a1.tsv', header=T)
pheno <- fread('/nfs/research/birney/users/tomas/gene_modules/cleaned_and_linked_phenotypes/asthma_icd10_J45_EFO_0000270-2_7.txt', header=T, col.names = c('eid', 'y'))
dtype <- 'binary'

In [50]:
geno2 <- fread('/nfs/research/birney/projects/association/gene_burden/genotypes/mask1_pLoF_canonical_rare_v1/ukb23156.mask1.c22.gene_burden.tsv', header=T)

print(length(intersect(colnames(geno), colnames(geno2))))

[1] 200643


In [57]:
pheno <- merge(pheno, covar, by='eid')
pheno[, eid:=as.character(eid)]
setkey(pheno, eid)
head(pheno)

eid,y,batch,sex,age,f.22009.0.1,f.22009.0.2,f.22009.0.3,f.22009.0.4,f.22009.0.5,f.22009.0.6,f.22009.0.7,f.22009.0.8,f.22009.0.9,f.22009.0.10,age2,sex_age,sex_age2
<chr>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>
1000040,1,1,1,62,-11.1732,5.37414,-1.76331,0.999203,-4.46673,-0.915626,1.4250400,-0.900987,-1.205530,3.360040,3844,62,3844
1000071,0,1,1,54,-13.8896,4.77904,-4.00437,4.648340,4.32980,2.237950,0.8034460,3.650780,-2.878140,0.260571,2916,54,2916
1000088,0,0,0,68,-13.2771,7.10413,-3.61512,3.301260,3.54118,-0.453903,0.7269860,0.601553,5.994300,0.464687,4624,0,0
1000109,0,1,0,50,-11.1648,2.79825,-3.87874,5.902440,-3.77853,1.589280,-2.3361400,-1.153620,3.806320,0.348935,2500,0,0
1000132,0,1,1,58,-14.5572,3.55023,2.60868,-0.323878,-4.78660,-2.834320,-1.5133600,1.255380,3.187980,-0.502443,3364,58,3364
1000241,0,0,0,43,-13.7094,4.50956,-2.27556,4.316810,-1.14060,0.995365,0.0948874,-0.994825,0.685751,1.681580,1849,0,0


In [58]:
keep <- intersect(pheno$eid, colnames(geno))
print(length(keep))

[1] 197099


In [59]:
pheno <- pheno[keep, ]
dim(pheno)
head(pheno)

[1] 197099     18

eid,y,batch,sex,age,f.22009.0.1,f.22009.0.2,f.22009.0.3,f.22009.0.4,f.22009.0.5,f.22009.0.6,f.22009.0.7,f.22009.0.8,f.22009.0.9,f.22009.0.10,age2,sex_age,sex_age2
<chr>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>
1000040,1,1,1,62,-11.1732,5.37414,-1.76331,0.999203,-4.46673,-0.915626,1.4250400,-0.900987,-1.205530,3.360040,3844,62,3844
1000071,0,1,1,54,-13.8896,4.77904,-4.00437,4.648340,4.32980,2.237950,0.8034460,3.650780,-2.878140,0.260571,2916,54,2916
1000088,0,0,0,68,-13.2771,7.10413,-3.61512,3.301260,3.54118,-0.453903,0.7269860,0.601553,5.994300,0.464687,4624,0,0
1000109,0,1,0,50,-11.1648,2.79825,-3.87874,5.902440,-3.77853,1.589280,-2.3361400,-1.153620,3.806320,0.348935,2500,0,0
1000132,0,1,1,58,-14.5572,3.55023,2.60868,-0.323878,-4.78660,-2.834320,-1.5133600,1.255380,3.187980,-0.502443,3364,58,3364
1000241,0,0,0,43,-13.7094,4.50956,-2.27556,4.316810,-1.14060,0.995365,0.0948874,-0.994825,0.685751,1.681580,1849,0,0


In [60]:
geno_keep = as.matrix(geno[, -1])[,keep]
rownames(geno_keep) <- geno$Gene
dim(geno_keep)

[1]    152 197099

In [61]:
res <- matrix(NA, nrow=nrow(geno), ncol=4, dimnames = list(rownames(geno_keep), c('est', 'stderr', 'tval', 'pval')))
# loop over genes
for (g in geno$Gene) {
    pheno[, x:=geno_keep[g, ]]
    if (dtype == 'quant') fit <- lm(data = pheno[, -1], formula = y ~ .)
    if (dtype == 'binary') fit <- glm(data = pheno[, -1], formula = y ~ ., family = 'binomial')
    coef_tb <- coef(summary(fit))
    if ('x' %in% rownames(coef_tb)) res[g,] <- coef_tb['x', ]
}

In [18]:
fwrite(as.data.table(res, keep.rownames=T), 'res.tsv', sep='\t')


In [36]:
coef(summary(fit), complete = T)

,Estimate,Std. Error,t value,Pr(>|t|)
(Intercept),0.4237784938,8.223788e-02,5.1530814,2.564870e-07
batch,0.0185021664,2.892814e-03,6.3959058,1.599539e-10
sex,0.6040328847,1.215396e-01,4.9698455,6.706228e-07
age,-0.0180045535,3.022942e-03,-5.9559711,2.589680e-09
f.22009.0.1,-0.0002704239,2.299307e-05,-11.7611032,6.346258e-32
f.22009.0.2,0.0003753333,4.341992e-05,8.6442659,5.454743e-18
f.22009.0.3,-0.0005223554,8.099107e-05,-6.4495431,1.124461e-10
f.22009.0.4,0.0003242084,1.188198e-04,2.7285728,6.361469e-03
f.22009.0.5,0.0006797330,1.663922e-04,4.0851246,4.407057e-05
f.22009.0.6,-0.0004970257,2.446757e-04,-2.0313652,4.221931e-02


In [25]:
pheno <- fread('/nfs/research/birney/users/tomas/gene_modules/cleaned_and_linked_phenotypes/cancer_ukb_f_id_40009_EFO_0000311-8.txt', header=T, col.names = c('eid', 'y'))
table(pheno$y)


     0      1      2      3      4      5      6      7      8     11     12 
412725  67035  18078   3515    852    195     67     14      7      1      1 
    17 
     1 

In [63]:
geno[1:10,1:10]

Gene,2232711,5433064,4840998,3657113,5827994,1977222,3621370,1679063,2735599
<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
CHEBI_9150,0,0,0,0,0,0,0,0,0
EFO_0000270,0,0,0,0,0,0,0,0,0
EFO_0000274,0,0,0,0,0,0,0,0,0
EFO_0000305,0,0,0,0,0,0,0,0,0
EFO_0000311,0,0,0,0,0,0,0,0,0
EFO_0000378,0,0,0,0,1,0,0,0,0
EFO_0000384,0,0,1,1,1,0,1,0,0
EFO_0000676,0,0,0,0,0,0,1,0,0
EFO_0000685,0,0,1,0,0,0,0,0,0
